# Imports and Constants

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline
import random
import os

from savingOutputs import *
from loadData import *
from masks import *
from decoding import *
from plots import *

In [2]:
SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
length = nb_runs * len(classes)
subjects_ids = range(1, 24)  # TODO modify this to get all subjects (here we only have 1 for the example)
n_subjects = len(subjects_ids)
n_perms = 1000
class_labels = [""] * (2 * length)
for i in range(length * 2):
    class_labels[i] = classes[(i // nb_runs) % len(classes)]
labels = dict()
labels["vis"] = np.array(class_labels[:length])
labels["aud"] = np.array(class_labels[length:])

classical_tasks_regions = [(["vis"], ["V5_R", "V5_L"]),
                           (["aud"], ["PT_R", "PT_L"]),
                           (["aud"], ["V5_R", "V5_L"])]

cross_modal_task_regions = [(["vis", "aud"], ["V5_R", "V5_L"])]

scaler = sklearn.preprocessing.StandardScaler()
classifier = sklearn.svm.SVC(C = 1, kernel = 'linear', random_state=SEED)  # other parameters are default parameters
pipeline = Pipeline([('std', scaler), ('svm', classifier)])
decoder = Decoder(n_perm=n_perms, model=pipeline, n_classes=len(classes), n_splits=nb_runs, seed=SEED)

# Loading data

In [4]:
cv_scores = [dict() for _ in subjects_ids]
p_values = [dict() for _ in subjects_ids]
scores_perms = [dict() for _ in subjects_ids]
maps_masked = [dict() for _ in subjects_ids]

In [ ]:
for i, subj_id in enumerate(subjects_ids):
    t_maps, beta_maps = get_maps([subj_id])
    masks = get_masks([subj_id], plot=False)
    maps = apply_mask_to_maps(t_maps, masks)
    maps_masked[i]["vis"] = get_part_of_maps(maps, 0, length)  # maps acquired for the vision experiment
    maps_masked[i]["aud"] = get_part_of_maps(maps, length, 2 * length)  # maps acquired for the audition experiment
    del t_maps ; del beta_maps ; del masks  # to relieve memory
    print("Loading done for subject "+str(subj_id)+"/"+str(n_subjects))

# Within-modality decoding

In [ ]:
for i, subj_id in enumerate(subjects_ids):
    # within-modality decoding : training on a task and decoding on other samples from same task
    for tasks, regions in classical_tasks_regions:
        cv_sc, p_val, scores_perm = decoder.classify_tasks_regions(maps_masked[i], labels, tasks, regions)
        cv_scores[i].update(cv_sc)
        p_values[i].update(p_val)
        scores_perms[i].update(scores_perm)
    print("Within-modality decoding done for subject "+str(subj_id)+"/"+str(n_subjects))



# Cross-modal decoding

In [ ]:
for i, subj_id in enumerate(subjects_ids):
    # cross-modal decoding : training on a task and decoding on samples from another task
    for tasks, regions in cross_modal_task_regions:
        scores_cross_mod = decoder.cross_modal_decoding(maps_masked[i], labels, tasks, regions)
        cv_scores[i].update(scores_cross_mod)

    print("Cross-modal decoding done for subject "+str(subj_id)+"/"+str(n_subjects))

# Saving results

In [9]:
create_directory("out")
save_dicts("cv_scores.csv", cv_scores, list(cv_scores[0].keys()), subjects_ids)
save_dicts("p_values.csv", p_values, list(p_values[0].keys()), subjects_ids)
save_dicts_perms("scores_perms.csv", scores_perms, subjects_ids)

# Plotting results (from files of saved results)

In [3]:
plt_directory = "plots"
create_directory(plt_directory)
plotter = Plotter(plt_directory, subjects_ids)
plotter.plot_cv_pval("out/cv_scores.csv", "cv score", plotter.cv_scores_dir, chance_level = True)
plotter.plot_cv_pval("out/p_values.csv", "p-value", plotter.p_values_dir)
plotter.plot_perms_scores("out/scores_perms.csv", n_perms)

# Visualizing differences between audition and vision in the voxels of a ROI

In [ ]:
region = "V5_R"
n_voxels = maps_masked[0]["vis"][0][region].shape[1]
mean_aud = np.zeros(n_voxels)
mean_vis = np.zeros(n_voxels)

for i in range(n_subjects):
    mean_vis += np.mean(maps_masked[i]["vis"][0][region],axis=0)/n_subjects
    mean_aud += np.mean(maps_masked[i]["aud"][0][region],axis=0)/n_subjects

plt.plot(range(n_voxels),mean_vis,label = "vision")
plt.plot(range(n_voxels),mean_aud,label = "audition")
plt.xlabel("voxel id")
plt.ylabel("intensity")
plt.title("Average voxel intensities for right V5")
plt.legend()
plt.savefig("plots/why_normalize_cross_modal.png")